# Quantum Amplitude Estimation

Quantum amplitude amplification was an algorithm to amplify the amplitude of a certain quantum state.  

Quantum amplitude estimation, as the name implies, is an algorithm for estimating the amplitude of a certain quantum state. Such an operation can be performed using the same oracle as quantum amplitude amplification.

The following diagram used in the explanation of amplitude amplification is used again.

<img width="30%" src="https://upload.wikimedia.org/wikipedia/commons/1/16/Grovers_algorithm_geometry.png">

Ref: https://en.wikipedia.org/wiki/Grover%27s_algorithm

Consider performing an amplitude estimation for $\lvert s\rangle$.  
Rewrite $\lvert s\rangle$ as follows.

$$
\lvert s\rangle = \sqrt{1-a} \lvert \omega_{\perp}\rangle - \sqrt{a}\lvert\omega\rangle\ \ \ \ (\lvert \omega_{\perp}\rangle = \lvert s'\rangle)
$$

We want to find the amplitude of $\lvert \omega\rangle$, $\sqrt{a}$.

$U_{\omega}$ and $U_s$ can be written respectively as follows.

$$
U_{\omega} = I - 2\lvert \omega \rangle \langle \omega \rvert \\
U_{s} = 2\lvert s \rangle \langle s \rvert - I
$$

The oracle $U_s U_{\omega}$ used for amplification has the following eigenvector $\lvert\psi_{\pm}\rangle$ and eigenvalue $\lambda_{\pm}$.

$$
\lvert\psi_{\pm}\rangle = \frac{1}{\sqrt{2}} (\lvert\omega_{\perp}\rangle \mp i\lvert\omega\rangle) \\
\lambda_{\pm} = e^{i2\theta_a}\ \ \ \ (\sin{\theta_a} = \sqrt{a})
$$

Once you have $\theta_a$, you can find the amplitude $\sqrt{a}$ of $\lvert \omega\rangle$.  
Here, we use quantum phase estimation to find $\theta_a$.  
Quantum phase estimation is an algorithm to find the $\theta$ (phase) of a unitary matrix (in this case $U_s U_{\omega}$) at its eigenvalue $e^{i\theta}$. (See the tutorial on quantum phase estimation for more details)

Quantum phase estimation requires the preparation of eigenstates (or their approximations) corresponding to the eigenvalues we want to find.  
Fortunately, the state $\lvert s\rangle$ can be written as a superposition of two eigenstates.

$$
\lvert s\rangle = \frac{1}{\sqrt{2}} \bigl(e^{i\theta_a} \lvert \psi_{+}\rangle + e^{-i\theta_a} \lvert \psi_{-}\rangle\bigr)
$$

With quantum phase estimation, the phase of the eigenvalue corresponding to either eigenstate can be obtained probabilistically.  
In this case, we get $2\theta_a$ or $-2\theta_a$, but the final value we want to find is $\sin{\theta_a} = \sqrt{a}$.  
From $\sin (\theta_a) = \sin (-\theta_a)$, we can estimate the amplitude in both cases.

Thus, if we have the state $\lvert s\rangle$ and the amplitude amplification oracle for it, we can estimate the amplitude.  
The following is the algorithm procedure.

1. Prepare the state $\lvert s\rangle$ for which you want to estimate the amplitude.
2. Prepare the oracle $U_s U_{\omega}$.
3. Find the eigenvalues of $U_s U_{\omega}$ by quantum phase estimation and calculate the amplitude you want to estimate.

Let's implement this in blueqat.

Consider a two-qubit example similar to the amplitude amplification tutorial.  
There are four states of a two-qubit: 00, 01, 10, and 11. We would like to estimate the amplitude of a particular state among them.  

In [8]:
from blueqat import Circuit
import numpy as np

In order to perform quantum phase estimation in the following, we will prepare a control gate version of the oracle used in amplitude amplification.

In [9]:
Us = Circuit(2).h[:].x[:].cz[0,1].x[:].h[:]
Uw00 = Circuit(2).s[:].cz[0,1].s[:]
Uw01 = Circuit(2).s[1].cz[0,1].s[1] 
Uw10 = Circuit(2).s[0].cz[0,1].s[0]
Uw11 = Circuit(2).cz[0,1]

def C_Us(qc, c, t1, t2):
    qc.ch[c, t1].ch[c, t2].cx[c, t1].cx[c, t2]
    qc.cx[t1, t2].tdg[t2].cx[c, t2].t[t2].cx[t1, t2].tdg[t2].cx[c, t2].t[t1].t[t2].cx[c, t1].t[c].tdg[t1].cx[c, t1]
    qc.cx[c, t1].cx[c, t2].ch[c, t1].ch[c, t2]
    
def C_Uw00(qc, c, t1, t2):
    qc.cphase(np.pi/2)[c, t1].cphase(np.pi/2)[c, t2]
    qc.cx[t1, t2].tdg[t2].cx[c, t2].t[t2].cx[t1, t2].tdg[t2].cx[c, t2].t[t1].t[t2].cx[c, t1].t[c].tdg[t1].cx[c, t1]
    qc.cphase(np.pi/2)[c, t1].cphase(np.pi/2)[c, t2]
    
def C_Uw01(qc, c, t1, t2):
    qc.cphase(np.pi/2)[c, t2]
    qc.cx[t1, t2].tdg[t2].cx[c, t2].t[t2].cx[t1, t2].tdg[t2].cx[c, t2].t[t1].t[t2].cx[c, t1].t[c].tdg[t1].cx[c, t1]
    qc.cphase(np.pi/2)[c, t2]
    
def C_Uw10(qc, c, t1, t2):
    qc.cphase(np.pi/2)[c, t1]
    qc.cx[t1, t2].tdg[t2].cx[c, t2].t[t2].cx[t1, t2].tdg[t2].cx[c, t2].t[t1].t[t2].cx[c, t1].t[c].tdg[t1].cx[c, t1]
    qc.cphase(np.pi/2)[c, t1]
    
def C_Uw11(qc, c, t1, t2):
    qc.cx[t1, t2].tdg[t2].cx[c, t2].t[t2].cx[t1, t2].tdg[t2].cx[c, t2].t[t1].t[t2].cx[c, t1].t[c].tdg[t1].cx[c, t1]

The eigenvalues of the amplitude amplification oracle are then obtained by quantum phase estimation.

In [10]:
import math

# 逆量子フーリエ変換関数を用意
def qft_rotate_single_inv(circuit, i, n):
    if n == 0:
        return circuit
    for qubit in range(0, i):
        circuit.cphase(-np.pi/2**(i - qubit))[n - 1 - qubit, n - 1 - i]
    circuit.h[n - 1 - i]

def qft_dagger(circuit, n):
    for i in range(math.floor(n/2)):
        circuit.swap[i, n - (i + 1)]
    for i in range(n):
        qft_rotate_single_inv(circuit, i, n)

In [15]:
n_encode = 4 # 求めたい固有値の位相角をエンコードする量子ビット数
n_eigstate = 2 # 固有状態の量子ビット数
n = n_encode + n_eigstate

qc = Circuit(n)
qc.h[n_encode].h[n_encode + 1] # 固有状態を用意

for qubit in range(n_encode):
    qc.h[qubit]

repetitions = 1
for count in reversed(range(n_encode)):
    for i in range(repetitions):
        C_Uw00(qc, count, n_encode, n_encode + 1)
        C_Us(qc, count, n_encode, n_encode + 1)
        #qc.cp(theta, count, n_encode)
    repetitions *= 2

qft_dagger(qc, n_encode)

for n in range(n_encode):
    qc.m[n]

In [22]:
res = qc.run(shots = 1024)
for key in res.keys():
    print(key[:n_encode], ':', res[key])

1101 : 342
0010 : 90
0011 : 371
1100 : 26
1110 : 87
0100 : 21
0110 : 8
1111 : 16
0000 : 10
1001 : 3
1010 : 8
0111 : 6
0001 : 13
0101 : 14
1011 : 9


This time the amplitude estimated state is an equal superposition of four states.Therefore, each state ($\lvert 00\rangle$, $\lvert 01\rangle$, $\lvert 10\rangle$, $\lvert 11\rangle$) has an amplitude of $1/2$.  

Thus, in the initial state $\lvert s\rangle = \sqrt{1-a}\lvert \omega_{\perp}\rangle - \sqrt{a}\lvert\omega\rangle$, $\sqrt{a} = 1/2$ and $\theta_a = \pi / 6$.

Since the oracle used for amplification has the eigenvalue $\exp(\pm i 2\theta) = \exp(\pm i\pi / 3)$, $\frac{\pm\pi/3}{2\pi} = \pm\frac{1}{6}$ is the output expected from quantum phase estimation.

In the example above, the output of the quantum phase estimation is encoded in 4 qubits, so the output is obtained with 4-bit precision.  
In the measurements, '0011' represents $\frac{3}{2^{4}} = 0.1875$, which is the closest value to $\frac{1}{6} = 0.166\cdots$.  
This gives us an approximation of $\theta$.  

From the above, we were able to perform quantum amplitude estimation.